In [1]:
from hdmf.common import ExternalResources
from hdmf.common import DynamicTable

from hdmf import Data
import pandas as pd

er = ExternalResources(name='example')

/Users/mavaylon/Research/hdmf_testing/hdmf/src/hdmf/container.py:181: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


# Using the add_ref method

add_ref will be one of the ExternalResources class methods that simplifies how to add new data.
You can think of add_ref as a wrapper function.

The function takes in the keyword arguments and will automatically create
new entries into tables that don't have the inputs as existing data.
In our example below, all of our Tables were empty before add_ref,
so calling the method will create new rows of data in every table
mentioned prior.
Note: If you want to reference objects then you need to reference the id
as a string, but if you want to reference a key then you reference
the key object itself.

In [2]:
data = Data(name="species", data=['Homo sapiens', 'Mus musculus'])
er.add_ref(container=data, field='', key='Homo sapiens', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid9606',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')

er.add_ref(container=data, field='', key='Mus musculus', resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid10090',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

(<hdmf.common.resources.Key at 0x7f96f30c5110>,
 <hdmf.common.resources.Entity at 0x7f96f30c50d0>)

In [3]:
er.resources.to_dataframe()

,resource,resource_uri
0,NCBI_Taxonomy,https://www.ncbi.nlm.nih.gov/taxonomy


# Using the add_ref method with a field

In the above example, field was empty because the data of the Data object was
being associated with a resource. The Data object may also have attributes
that you would like to associate with resources.

To disambiguate between these different fields, you can set the 'field' keyword
argument corresponding to the name of the dataset or attribute containing the
values being associated with a resource. For example:

In [4]:
genotypes = DynamicTable(name='genotypes', description='My genotypes')
genotypes.add_column(name='genotype_name', description="Name of genotypes")
genotypes.add_row(id=0, genotype_name='Rorb')
er.add_ref(container=genotypes, field='genotype_name', key='Mus musculus', resource_name='MGI Ontology',
           resource_uri='http://www.informatics.jax.org/', entity_id='MGI:1346434',
           entity_uri="http://www.informatics.jax.org/probe/key/804614")

(<hdmf.common.resources.Key at 0x7f96f310c490>,
 <hdmf.common.resources.Entity at 0x7f96f310c5d0>)

# Using the get_keys method

In [5]:
# All Keys
er.get_keys()

,key_name,resources_idx,entity_id,entity_uri
0,Homo sapiens,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,Mus musculus,0,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
2,Mus musculus,1,MGI:1346434,http://www.informatics.jax.org/probe/key/804614


In [5]:
# Single Key
er.get_keys(keys=er.get_key('Homo sapiens'))

,key_name,resources_idx,entity_id,entity_uri
0,Homo sapiens,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


# Using the add_key and the get_key methods

Mentioned in the note aobve, the user needs to reference a key object
when referencing a key.

In [3]:
# Let's start with a new instance of ExternalResources
er = ExternalResources(name='new_example')

# The add_key method will create a new key
er.add_key(key_name='Homo sapiens')
er.add_key(key_name='Mus musculus')

# The get_key method will return the key object of the key_name.
key_object = er.get_key(key_name='Mus musculus')

In [7]:
# We can use the key_object to reference keys.
er.add_ref(container=data, field='', key=key_object, resource_name='NCBI_Taxonomy',
           resource_uri='https://www.ncbi.nlm.nih.gov/taxonomy', entity_id='NCBI:txid10090',
           entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

It is also useful to associate an existing key with an additional entity.
For example, we can reference the Mus musculus key for a different entity from
a new resource.

In [8]:
er.add_ref(container=data, field='', key=key_object, resource_name='ITIS',
           resource_uri='https://www.itis.gov/', entity_id='180366',
           entity_uri='https://www.itis.gov/servlet/SingleRpt/SingleRpt?search_topic=TSN&search_value=180366')

er.get_keys()

,key_name,resources_idx,entity_id,entity_uri
0,Mus musculus,0,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,Mus musculus,1,180366,https://www.itis.gov/servlet/SingleRpt/SingleR...


In [9]:
# This will return a pandas dataframe of the KeyTable
er.keys.to_dataframe()

,key
0,Homo sapiens
1,Mus musculus


# Using the add_keys method

This is another wrapper function, but allows the use of a pandas DataFrame
to add/reference keys, resources, and entities.

In [11]:
new_data = {
    'key_name': 'Homo sapiens',
    'resources_idx': 0,
    'entity_id': 'NCBI:txid9606',
    'entity_uri': 'https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606'}

df = pd.DataFrame(new_data, index=[0])

er.add_keys(df)

{'Homo sapiens': <hdmf.common.resources.Key at 0x7fe68a920c90>}

# Using the add_resource method

In [12]:
er = ExternalResources(name='new_example')


er.add_resource(resource='NCBI_Taxonomy', uri='https://www.ncbi.nlm.nih.gov/taxonomy')

# This will return a pandas dataframe of the ResourceTable
er.resources.to_dataframe()

/Users/mavaylon/Research/hdmf_testing/hdmf/src/hdmf/container.py:181: UserWarning: ExternalResources is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))


,resource,resource_uri
0,NCBI_Taxonomy,https://www.ncbi.nlm.nih.gov/taxonomy


# Using the add_entity method

In [13]:
er.add_entity(key='Homo sapiens', resources_idx=0,
              entity_id='NCBI:txid9606',
              entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=9606')

er.add_entity(key='Mus musculus', resources_idx=0,
              entity_id='NCBI:txid10090',
              entity_uri='https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=10090')

# This will return a pandas dataframe of the EntityTable
er.entities.to_dataframe()

,keys_idx,resources_idx,entity_id,entity_uri
0,0,0,NCBI:txid9606,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...
1,1,0,NCBI:txid10090,https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/...


##### (Mention get_resource)

# Using the add_object method

In [14]:
er.add_object(container=data, field='Data')

# This will return a pandas dataframe of the ObjectTable
er.objects.to_dataframe()

,object_id,field
0,6db26450-cef1-49b9-8dc7-4997ec3db02b,Data


# Using the add_external_reference method

In [16]:
# Let's create a new Data instance

data_mouse = Data(name="species", data='Mus musculus')

In [17]:
# This method below returns an Object object, but will also create one if it doesn't exist.
object_ = er._check_object_field(data_mouse, field='')

In [18]:
key_object = er.get_key(key_name='Mus musculus')

er.add_external_reference(object_, key_object)

# This return ObjectKeyTable
er.object_keys.to_dataframe()

,objects_idx,keys_idx
0,1,1
